This notebook contains the initial experimentation code for generating the final summary of articles retrieved for a given image using the bing visual search api. This is done for the test data (first 1k samples). These summaries will be stored to provide initial context for the debating models.

In [1]:
import sys
sys.path.append("..")

import http.client, urllib.parse
import requests
import json
import os.path

BASE_URI = "https://api.bing.microsoft.com/v7.0/images/visualsearch"
SUBSCRIPTION_KEY = 'fcbb57be03f64cf191aeb1ba7835fc04'

from utils.data import get_data
from utils.external_retrieval import get_summary, get_webpage_text

In [2]:
def get_urls(file):
    matching_urls = []
    # Construct the request headers
    headers = {
        "Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY
    }

    # Send the POST request
    response = requests.post(
        BASE_URI,
        headers=headers,
        files=file
    )

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response data
        data = response.json()
        #print(data)
        result_list = []
        if len(data['tags'][0]['actions'][0]['data']['value']) == 0:
            result_list = data['tags'][0]['actions'][2]['data']['value']
        else:
            result_list = data['tags'][0]['actions'][0]['data']['value']
        for res in result_list:
            if len(matching_urls) == 3:
                break
            else:
                matching_urls.append(res['hostPageUrl'])
        return matching_urls
    else:
        # Handle the error
        return matching_urls
        #print(f"Error: {response.status_code} - {response.text}")

In [44]:
for i in range(1):
    image, caption, image_path, annotation = get_data(i)
    file = {'image' : ('myfile', open(image_path, 'rb'))}
    matching_urls = get_urls(file)
    print("matching_urls: ", matching_urls)
    search_result = get_summary(matching_urls)

matching_urls:  ['https://www.theconstructionindex.co.uk/news/view/mps-call-for-government-action-on-blacklisting', 'https://socialistworker.co.uk/news/tens-of-thousands-more-names-could-have-been-on-blacklists/', 'https://www.theguardian.com/law/2021/may/27/recruiters-urged-to-clamp-down-on-exploitation-via-umbrella-companies']


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Found 3 search results.


In [3]:
matching_urls = ['https://www.theconstructionindex.co.uk/news/view/mps-call-for-government-action-on-blacklisting', 'https://socialistworker.co.uk/news/tens-of-thousands-more-names-could-have-been-on-blacklists/', 'https://www.theguardian.com/law/2021/may/27/recruiters-urged-to-clamp-down-on-exploitation-via-umbrella-companies']

In [4]:
search_result = get_summary(matching_urls)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Found 3 search results.


In [5]:
print(search_result)

SUMMARY:
                    - The House of Commons Scottish Affairs committee has been investigating the use of the Consulting Association by major contractors to prevent suspected "troublemakers" from getting jobs in the industry.
                    - The committee has described the practice as an "organized conspiracy" by big construction firms to discriminate against workers who raised legitimate grievances over health and safety and other industrial issues.
                    - The committee has received written evidence and held a formal committee hearing on the 3rd of July with Gail Cartmail of Unite the Union, who has given what they believe to be clear proof that blacklisting for trade union and health and safety activities has been going on within the Crossrail project.
                    - The Consulting Association was shut down by the Information Commissioner in 2009.
                    - The scale of the blacklist may be far larger than previously thought and goes wel

In [9]:
texts = []
for matching_url in matching_urls:
    texts.append(get_webpage_text(matching_url))
text = ""
print("Found {} search results.".format(len(texts)))
for i in range(len(texts)):
    #only take top k=3 articles
    if i == 3: 
        break
    if "the" not in texts[i]:
        #naive way to ensure text is in English
        continue
    text += "\n\n"
    text += texts[i]

Found 3 search results.


In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def generate():
  vertexai.init(project="thesis-426010", location="us-central1")
  model = GenerativeModel(
    "gemini-1.5-flash-001",
  )
  responses = model.generate_content(
      [text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")

text1 = """Provide a brief summary for the following article:

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in \"Harry Potter and the Order of the Phoenix\" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. \"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,\" he told an Australian interviewer earlier this month. \"I don\'t think I\'ll be particularly extravagant. \"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs.\" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film \"Hostel: Part II,\" currently six places below his number one movie on the UK box office chart. Details of how he\'ll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. \"I\'ll definitely have some sort of party,\" he said in an interview. \"Hopefully none of you will be reading about it.\" Radcliffe\'s earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. \"People are always looking to say \'kid star goes off the rails,\'\" he told reporters last month. \"But I try very hard not to go that way because it would be too easy for them.\" His latest outing as the boy wizard in \"Harry Potter and the Order of the Phoenix\" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter\'s latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called \"My Boy Jack,\" about author Rudyard Kipling and his son, due for release later this year. He will also appear in \"December Boys,\" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer\'s \"Equus.\" Meanwhile, he is braced for even closer media scrutiny now that he\'s legally an adult: \"I just think I\'m going to be more sort of fair game,\" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."""

generation_config = {
    "max_output_tokens": 768,
    "temperature": 0.2,
    "top_p": 0.8,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()

